## Cleaning

In [2]:
''' Clean the data and write to a CSV file'''

import csv

# Open the original CSV file and the new CSV file
with open('parsed_data/parsed_ig_dms.csv', 'r', newline='', encoding='utf-8') as input_file, \
     open('intermediate_data/cleaned_ig_dms.csv', 'w', newline='', encoding='utf-8') as output_file:

    reader = csv.reader(input_file)
    writer = csv.writer(output_file)

    # Write the header row to the new CSV file
    headers = next(reader)
    writer.writerow(headers)

    # Iterate over the rows in the original CSV file
    for row in reader:
        # If the content contains "sent an attachment", replace the content with None
        if "sent an attachment" in row[2]:
            row[2] = None
        writer.writerow(row)